# Exemplo de Jogo Real

Este foi o código que utilizei para demonstrar em um jogo real como ele pode ser aplicado, auxiiando o jogador para descobrir a resposta mais rapidamente e fazer palpites mais sensatos (o código não foi utilizado durante o jogo real).

_OBS1: A ordem é jogador1, jogador2, jogador3, eassim por diante, porém muitas vezes os jogadores não estavam e sua vez era pulada._

_OBS2: Este jogo tinha sido preparado para 7 jogadores, porém faltou um jogador, então suas cartas foram reveladas a todos {cartas_fora}._

In [29]:
class Clue():
    def __init__(self, lista_geral,
                 jogadores, dicionario,
                 possibilidades, certezas,
                 eu, cartas_fora, tipos):

        self.general = lista_geral
        self.players = jogadores
        self.dict = dicionario
        self.possibilities = possibilidades
        self.correct = certezas
        self.me = eu
        self.myidx = jogadores.index(eu)
        self.out_of_game = cartas_fora
        self.types = tipos
        self.results = [None, None, None]

    def palpite(self, jogador, pessoa, lugar, arma):
        cartas = [pessoa, lugar, arma]
        i = self.players.index(jogador)
        cartas_originais = []

        # Remove as cartas que já não estão no dicionário
        for c in cartas:
            if c not in self.dict.keys():
                cartas.remove(c)
            cartas_originais.append(c)

        # i = jogador que deu o palpite
        # j = jogador a responder o palpite
        if i == len(self.players)-1:
            j = 0
        else:
            j = i+1

        # Pergunta para todos os outros jogadores, em ordem, se eles possuem
        # alguma das cartas no palpite
        y = ''
        while j != i or y != 's':
            if j == i:
                break

            if j == len(self.players):
                j = 0

            y = input(f'Jogador(a) {self.players[j]} tem alguma dessas cartas(s/n)? ')

            if y == 'n':
                # Se a resposta for 'não', remove o jogador perguntado da lista
                # de possibilidades de cada carta no palpite
                if self.players[j] != self.me:
                    for chave, valores in self.dict.items():
                        if chave in [pessoa, arma, lugar] and self.players[j] in valores:
                            l = valores.copy()
                            l.remove(self.players[j])
                            self.dict[chave] = l
            # Caso a resposta seja 'sim'
            elif y == 's':
                # Se o jogador perguntado for eu, passa
                if self.players[j] == self.me:
                    pass

                if self.players[i] != self.me:
                  boolean = 0
                  # Se alguma das cartas estiver na lista de certezas do
                  # jogador perguntado, para
                  for x in cartas_originais:
                      if x in self.correct[j]:
                          boolean = 1
                          break
                      # Se não, remove da lista de possibilidades
                      # as cartas que já temos certeza
                      if x in cartas:
                          if x in self.possibilities[self.myidx]:
                              cartas.remove(x)
                          if x in self.out_of_game:
                              cartas.remove(x)

                  # Adiciona as possibilidades à lista de possibilidades
                  if boolean != 1:
                      self.possibilities[j].append(cartas)
                  break
                else:
                  self.perguntaCarta(j)
                  break

            j += 1

        # Caso não haja nenhuma possibilidade para determinada carta
        # no dicionario, adiciona essa carta na lista de resultados
        # de acordo com o seu tipo
        chaves_a_remover = [chave for chave, valores in self.dict.items() if valores == []]

        for chave in chaves_a_remover:
            for l1 in self.possibilities:
                for l2 in l1:
                    if chave in l2:
                        l2.remove(chave)
                        for e in range(len(self.types)):
                            if chave in self.types[e]:
                                self.results[e] = chave

                    # Se houver uma lista com um elemento nas listas de possibilidades
                    # de um jogador, adiciona esse elemento na lista de certezas,
                    # caso já não esteja
                    if len(l2) == 1:
                        p = self.possibilities.index(l1)
                        if l2[0] not in self.correct[p]:
                            self.correct[p].append(l2[0])
                        l1.remove(l2)

        self.descobreCertezas()

        # Remove a lista de possibilidades de um jogador que contém um item correto
        # deste mesmo jogador
        for i in range(len(self.possibilities)):
            for p in self.possibilities[i]:
                for x in p:
                    if x in self.correct[i]:
                        self.possibilities[i].remove(p)

        # Eliminando a chave que já é uma certeza do dicionário
        keys_to_remove = []

        for chave in self.dict.keys():
          for i in range(len(self.correct)):
            if chave in self.correct[i]:
              keys_to_remove.append(chave)

        for chave in keys_to_remove:
            self.dict.pop(chave)



    # Percorre a lista de certezas e, caso haja uma certeza em alguma
    # lista de possibilidades, exclui a certeza
    def descobreCertezas(self):
        def remove_and_append(certeza, possibilidade):
            if len(possibilidade) == 1 and possibilidade[0] not in certeza:
                certeza.append(possibilidade[0])
                return True
            return False

        for i in range(len(self.correct)):
            if i != self.myidx:
                for possibilidade in self.possibilities[i][:]:
                    # Dentre as listas de possibilidades, há uma lista de tamanho 1,
                    # logo, é uma certeza
                    if remove_and_append(certeza=self.correct[i], possibilidade=possibilidade):
                        self.possibilities[i].remove(possibilidade)
                    else:
                        # Caso contrário, se um item da lista de possibilidades de um
                        # jogador estiver na lista de cetezas de outro jogador, remove
                        # este item das possibilidades
                        for j in range(len(self.correct)):
                            if j != i:
                                for certeza in self.correct[j][:]:
                                    if certeza in possibilidade:
                                        possibilidade.remove(certeza)
                                        if len(possibilidade) == 1:
                                            if possibilidade[0] not in certeza:
                                                self.correct[i].append(possibilidade[0])
                                            self.possibilities[i].remove(possibilidade)

    # Printa as respostas concretas descobertas (None para cada resultado que não há)
    def printResultados(self):
        print(f'Quem: {self.results[0]}')
        print(f'Onde: {self.results[1]}')
        print(f'Como: {self.results[2]}')

    # Pergunta a carta do jogador que disse 'sim', caso eu seja o jogador que
    # deu o palpite
    def perguntaCarta(self, j):
        c = input('Qual carta? ')
        for k in self.possibilities[j]:
            if c in k:
                self.possibilities[j].remove(k)

        for l1 in self.possibilities:
            for l2 in l1:
                if c in l2:
                    l2.remove(c)
        self.dict.pop(c)
        self.correct[j].append(c)

    # Printa as cartas descobertas de cada jogador
    def printCertezas(self):
        for i in range(len(self.correct)):
            print(f'Lista de certezas de {self.players[i]}: {self.correct[i]}\n')

    # Printa as possibilidades de cada jogador baseado nos 'sim' ditos
    def printPossibilidades(self):
        for i in range(len(self.possibilities)):
            if i != self.myidx:
                print(f'Lista de possibilidades de {self.players[i]}: {self.possibilities[i]}\n')

    # Printa as possibilidades de todas as cartas que não foram verificadas
    def printPossibCartasGeral(self):
        print('Lista de possiveis jogadores que tenham as cartas:\n')
        for x, y in self.dict.items():
            print(f'{x} - {self.dict[x]}')

    # Printa as possibilidades das cartas faltantes (o tipo que não está nos resultados)
    def printPossibCartasFaltantes(self):
        print('Lista de possiveis jogadores que tenham as cartas:\n')
        for i in range(len(self.results)):
            if self.results[i] is None:
                for x in self.types[i]:
                    if x in self.dict:
                        print(f'{x} - {self.dict[x]}')

In [30]:
# JOGO CLUE

jogadores = ['fora de jogo', 'jogador1', 'jogador2', 'jogador3', 'jogador4', 'jogador5', 'jogador6']

cartas_fora = ['mordomo alfred', 'sala de jogos', 'candelabro']

lista_geral = [['senhorita cherry', 'coronel banana', 'professor margarida', 'reverendo sapo',
                'cientista maluco', 'cozinheiro laranja', 'mordomo alfred'],
               ['escritorio', 'salao de trofeus', 'salao de jantar', 'biblioteca', 'sala de jogos',
                'sala de estar', 'jardim interno', 'cozinha', 'sala de musica', 'salao de festas'],
               ['candelabro', 'corda', 'cano', 'revolver', 'chave inglesa', 'veneno', 'faca']]

jogadores.remove('fora de jogo')

lista_geral[0].remove('mordomo alfred')
lista_geral[1].remove('sala de jogos')
lista_geral[2].remove('candelabro')

# Sou o jogador2
lista = []
lista.extend(lista_geral[0])
lista.extend(lista_geral[1])
lista.extend(lista_geral[2])

eu = 'jogador2'

j2_list = ['escritorio', 'cozinha', 'faca']

for x in j2_list:
    lista.remove(x)

dicionario = {}

j_possiveis = jogadores.copy()
j_possiveis.remove('jogador2')

for chave in lista:
    dicionario[chave] = j_possiveis

# listas de possibilidades
j1_list = []
# j2__list = ['escritorio', 'cozinha', 'faca']
j3_list = []
j4_list = []
j5_list = []
j6_list = []

possibilidades = [j1_list, j2_list, j3_list, j4_list, j5_list, j6_list]

# listas de certezas
j1_l = []
# j2_l = ['escritorio', 'cozinha', 'faca']
j3_l = []
j4_l = []
j5_l = []
j6_l = []

certezas = [j1_l, j2_list, j3_l, j4_l, j5_l, j6_l]

# Todos os outros jogaores tem a possibilidade de ter essas cartas
print(lista)

['senhorita cherry', 'coronel banana', 'professor margarida', 'reverendo sapo', 'cientista maluco', 'cozinheiro laranja', 'salao de trofeus', 'salao de jantar', 'biblioteca', 'sala de estar', 'jardim interno', 'sala de musica', 'salao de festas', 'corda', 'cano', 'revolver', 'chave inglesa', 'veneno']


## Rodada 1

In [31]:
clue = Clue(lista, jogadores, dicionario, possibilidades, certezas, eu, cartas_fora, lista_geral)

In [32]:
clue.palpite('jogador3', 'cientista maluco', 'sala de estar', 'chave inglesa')

Jogador(a) jogador4 tem alguma dessas cartas(s/n)? s


In [33]:
clue.palpite('jogador6', 'cozinha', 'revolver', 'cozinheiro laranja')

Jogador(a) jogador1 tem alguma dessas cartas(s/n)? s


In [34]:
clue.palpite('jogador4', 'coronel banana', 'chave inglesa', 'sala de estar')

Jogador(a) jogador5 tem alguma dessas cartas(s/n)? s


In [35]:
clue.printPossibilidades()

Lista de possibilidades de jogador1: [['revolver', 'cozinheiro laranja']]

Lista de possibilidades de jogador3: []

Lista de possibilidades de jogador4: [['cientista maluco', 'sala de estar', 'chave inglesa']]

Lista de possibilidades de jogador5: [['coronel banana', 'chave inglesa', 'sala de estar']]

Lista de possibilidades de jogador6: []



## Rodada 2

In [36]:
clue.palpite('jogador1', 'reverendo sapo', 'salao de festas', 'veneno')

Jogador(a) jogador2 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador3 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador4 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador5 tem alguma dessas cartas(s/n)? s


In [37]:
clue.palpite('jogador2', 'salao de jantar', 'chave inglesa', 'mordomo alfred')

Jogador(a) jogador3 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador4 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador5 tem alguma dessas cartas(s/n)? s
Qual carta? chave inglesa


In [38]:
clue.palpite('jogador3', 'cozinheiro laranja', 'salao de festas', 'veneno')

Jogador(a) jogador4 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador5 tem alguma dessas cartas(s/n)? s


In [39]:
clue.palpite('jogador4', 'professor margarida', 'revolver', 'salao de festas')

Jogador(a) jogador5 tem alguma dessas cartas(s/n)? s


In [40]:
clue.palpite('jogador6', 'salao de jantar', 'reverendo sapo', 'chave inglesa')

Jogador(a) jogador1 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador2 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador3 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador4 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador5 tem alguma dessas cartas(s/n)? s


In [41]:
clue.printPossibilidades()

Lista de possibilidades de jogador1: [['revolver', 'cozinheiro laranja']]

Lista de possibilidades de jogador3: []

Lista de possibilidades de jogador4: [['cientista maluco', 'sala de estar']]

Lista de possibilidades de jogador5: [['reverendo sapo', 'salao de festas', 'veneno'], ['cozinheiro laranja', 'salao de festas', 'veneno'], ['professor margarida', 'revolver', 'salao de festas']]

Lista de possibilidades de jogador6: []



In [42]:
clue.printCertezas()

Lista de certezas de jogador1: []

Lista de certezas de jogador2: ['escritorio', 'cozinha', 'faca']

Lista de certezas de jogador3: []

Lista de certezas de jogador4: []

Lista de certezas de jogador5: ['chave inglesa']

Lista de certezas de jogador6: []



## Rodada 3

In [43]:
clue.palpite('jogador1', 'cientista maluco', 'jardim interno', 'revolver')

Jogador(a) jogador2 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador3 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador4 tem alguma dessas cartas(s/n)? s


In [44]:
clue.palpite('jogador2', 'sala de musica', 'mordomo alfred', 'veneno')

Jogador(a) jogador3 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador4 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador5 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador6 tem alguma dessas cartas(s/n)? s
Qual carta? veneno


In [45]:
clue.palpite('jogador3', 'senhorita cherry', 'salao de jantar', 'veneno')

Jogador(a) jogador4 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador5 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador6 tem alguma dessas cartas(s/n)? s


In [46]:
clue.palpite('jogador4', 'reverendo sapo', 'faca', 'jardim interno')

Jogador(a) jogador5 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador6 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador1 tem alguma dessas cartas(s/n)? s


In [47]:
clue.palpite('jogador6', 'sala de musica', 'reverendo sapo', 'chave inglesa')

Jogador(a) jogador1 tem alguma dessas cartas(s/n)? s


In [48]:
clue.printPossibilidades()

Lista de possibilidades de jogador1: [['revolver', 'cozinheiro laranja']]

Lista de possibilidades de jogador3: []

Lista de possibilidades de jogador4: [['cientista maluco', 'sala de estar'], ['cientista maluco', 'revolver']]

Lista de possibilidades de jogador5: []

Lista de possibilidades de jogador6: []



In [49]:
clue.printCertezas()

Lista de certezas de jogador1: ['jardim interno', 'sala de musica']

Lista de certezas de jogador2: ['escritorio', 'cozinha', 'faca']

Lista de certezas de jogador3: []

Lista de certezas de jogador4: []

Lista de certezas de jogador5: ['chave inglesa', 'salao de festas']

Lista de certezas de jogador6: ['veneno']



In [50]:
clue.printResultados()

Quem: reverendo sapo
Onde: None
Como: None


In [51]:
clue.printPossibCartasGeral()

Lista de possiveis jogadores que tenham as cartas:

senhorita cherry - ['jogador1', 'jogador3', 'jogador6']
coronel banana - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
professor margarida - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
reverendo sapo - []
cientista maluco - ['jogador1', 'jogador4', 'jogador5', 'jogador6']
cozinheiro laranja - ['jogador1', 'jogador3', 'jogador5', 'jogador6']
salao de trofeus - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
salao de jantar - ['jogador6']
biblioteca - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
sala de estar - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
corda - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
cano - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
revolver - ['jogador1', 'jogador4', 'jogador5', 'jogador6']


In [52]:
clue.printPossibCartasFaltantes()

Lista de possiveis jogadores que tenham as cartas:

salao de trofeus - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
salao de jantar - ['jogador6']
biblioteca - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
sala de estar - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
corda - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
cano - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
revolver - ['jogador1', 'jogador4', 'jogador5', 'jogador6']


## Rodada 4

In [53]:
clue.palpite('jogador1', 'professor margarida', 'biblioteca', 'cano')

Jogador(a) jogador2 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador3 tem alguma dessas cartas(s/n)? s


In [54]:
clue.palpite('jogador3', 'sala de jantar', 'cano', 'cozinheiro laranja')

Jogador(a) jogador4 tem alguma dessas cartas(s/n)? s


In [55]:
clue.palpite('jogador4', 'reverendo sapo', 'faca', 'salao de trofeus')

Jogador(a) jogador5 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador6 tem alguma dessas cartas(s/n)? s


In [56]:
clue.palpite('jogador6', 'salao de jantar', 'senhorita cherry', 'cano')

Jogador(a) jogador1 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador2 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador3 tem alguma dessas cartas(s/n)? s


In [57]:
clue.printPossibilidades()

Lista de possibilidades de jogador1: [['revolver', 'cozinheiro laranja']]

Lista de possibilidades de jogador3: [['professor margarida', 'biblioteca', 'cano'], ['salao de jantar', 'senhorita cherry', 'cano']]

Lista de possibilidades de jogador4: [['cientista maluco', 'sala de estar'], ['cientista maluco', 'revolver'], ['cano', 'cozinheiro laranja']]

Lista de possibilidades de jogador5: []

Lista de possibilidades de jogador6: []



In [58]:
clue.printCertezas()

Lista de certezas de jogador1: ['jardim interno', 'sala de musica']

Lista de certezas de jogador2: ['escritorio', 'cozinha', 'faca']

Lista de certezas de jogador3: []

Lista de certezas de jogador4: []

Lista de certezas de jogador5: ['chave inglesa', 'salao de festas']

Lista de certezas de jogador6: ['veneno', 'salao de trofeus']



In [59]:
clue.printResultados()

Quem: reverendo sapo
Onde: None
Como: None


In [60]:
clue.printPossibCartasGeral()

Lista de possiveis jogadores que tenham as cartas:

senhorita cherry - ['jogador3', 'jogador6']
coronel banana - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
professor margarida - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
reverendo sapo - []
cientista maluco - ['jogador1', 'jogador4', 'jogador5', 'jogador6']
cozinheiro laranja - ['jogador1', 'jogador3', 'jogador5', 'jogador6']
salao de jantar - ['jogador6']
biblioteca - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
sala de estar - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
corda - ['jogador1', 'jogador3', 'jogador4', 'jogador5', 'jogador6']
cano - ['jogador3', 'jogador4', 'jogador5', 'jogador6']
revolver - ['jogador1', 'jogador4', 'jogador5', 'jogador6']


## Rodada 5

In [62]:
clue.palpite('jogador1', 'salao de trofeus', 'revolver', 'reverendo sapo')

Jogador(a) jogador2 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador3 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador4 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador5 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador6 tem alguma dessas cartas(s/n)? s


In [63]:
clue.palpite('jogador2', 'jardim interno', 'professor margarida', 'cano')

Jogador(a) jogador3 tem alguma dessas cartas(s/n)? s
Qual carta? professor margarida


In [64]:
clue.palpite('jogador3', 'revolver', 'coronel banana', 'salao de trofeus')

Jogador(a) jogador4 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador5 tem alguma dessas cartas(s/n)? s


In [65]:
clue.palpite('jogador4', 'escritorio', 'revolver', 'reverendo sapo')

Jogador(a) jogador5 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador6 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador1 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador2 tem alguma dessas cartas(s/n)? s


In [66]:
clue.palpite('jogador6', 'salao de jantar', 'mordomo alfred', 'revolver')

Jogador(a) jogador1 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador2 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador3 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador4 tem alguma dessas cartas(s/n)? n
Jogador(a) jogador5 tem alguma dessas cartas(s/n)? n


In [67]:
clue.printCertezas()

Lista de certezas de jogador1: ['jardim interno', 'sala de musica', 'cozinheiro laranja']

Lista de certezas de jogador2: ['escritorio', 'cozinha', 'faca']

Lista de certezas de jogador3: ['professor margarida']

Lista de certezas de jogador4: ['cientista maluco', 'cano']

Lista de certezas de jogador5: ['chave inglesa', 'salao de festas', 'coronel banana']

Lista de certezas de jogador6: ['veneno', 'salao de trofeus']



In [68]:
clue.printPossibilidades()

Lista de possibilidades de jogador1: []

Lista de possibilidades de jogador3: [['salao de jantar', 'senhorita cherry']]

Lista de possibilidades de jogador4: []

Lista de possibilidades de jogador5: []

Lista de possibilidades de jogador6: []



In [70]:
clue.printResultados()

Quem: reverendo sapo
Onde: None
Como: revolver


In [69]:
clue.printPossibCartasGeral()

Lista de possiveis jogadores que tenham as cartas:

senhorita cherry - ['jogador3', 'jogador6']
reverendo sapo - []
salao de jantar - ['jogador6']
biblioteca - ['jogador3', 'jogador4', 'jogador6']
sala de estar - ['jogador3', 'jogador4', 'jogador6']
corda - ['jogador3', 'jogador4', 'jogador6']
revolver - []


In [71]:
clue.printPossibCartasFaltantes()

Lista de possiveis jogadores que tenham as cartas:

salao de jantar - ['jogador6']
biblioteca - ['jogador3', 'jogador4', 'jogador6']
sala de estar - ['jogador3', 'jogador4', 'jogador6']


## Rodada 6

In [72]:
clue.palpite('jogador1', 'escritorio', 'coronel banana', 'faca')

Jogador(a) jogador2 tem alguma dessas cartas(s/n)? s


In [73]:
clue.palpite('jogador2', 'jardim interno', 'revolver', 'senhorita cherry')

Jogador(a) jogador3 tem alguma dessas cartas(s/n)? s
Qual carta? senhorita cherry


In [74]:
clue.printCertezas()

Lista de certezas de jogador1: ['jardim interno', 'sala de musica', 'cozinheiro laranja']

Lista de certezas de jogador2: ['escritorio', 'cozinha', 'faca']

Lista de certezas de jogador3: ['professor margarida', 'senhorita cherry']

Lista de certezas de jogador4: ['cientista maluco', 'cano']

Lista de certezas de jogador5: ['chave inglesa', 'salao de festas', 'coronel banana']

Lista de certezas de jogador6: ['veneno', 'salao de trofeus']



In [75]:
clue.printResultados()

Quem: reverendo sapo
Onde: None
Como: revolver


In [76]:
clue.printPossibCartasFaltantes()

Lista de possiveis jogadores que tenham as cartas:

salao de jantar - ['jogador6']
biblioteca - ['jogador3', 'jogador4', 'jogador6']
sala de estar - ['jogador3', 'jogador4', 'jogador6']


In [77]:
clue.printPossibCartasGeral()

Lista de possiveis jogadores que tenham as cartas:

reverendo sapo - []
salao de jantar - ['jogador6']
biblioteca - ['jogador3', 'jogador4', 'jogador6']
sala de estar - ['jogador3', 'jogador4', 'jogador6']
corda - ['jogador3', 'jogador4', 'jogador6']
revolver - []


O jogador3 fez uma acusação para as cartas {salao de jantar, revolver, reverendo sapo} e acertou, ganhando o jogo.